In [1]:
!pip install gradio torch pillow easyocr -q groq

In [2]:
import easyocr
import groq

reader = easyocr.Reader(['en'])
GROQ_API_KEY = "your_api_key"

client = groq.Client(api_key=GROQ_API_KEY)

def solve_puzzle(image):
  image_path = "image.png"
  image.save(image_path)

  results = reader.readtext(image_path)
  extracted_text = " ".join([res[1] for res in results])

  extracted_text = extracted_text.replace("??", "?")


  if "?" not in extracted_text:
    extracted_text += "?"

  print("Extracted text=" , extracted_text)

  refined_text = extracted_text.replace('x','*').replace('X','*').replace('=',' = ').strip()
  print("Refined text=", refined_text)


  puzzle_prompt = (f"You are an AI specializing in solving puzzles. Analyze the uploaded image, identify the math problem, solve it, and provide only the final answer in fewer than 2000 tokens."
                   f"\nPuzzle:\n {refined_text}\n"
                  "Format your response stricly as follows:\n"
                  "1. **Given equation**\n   -(original equations)\n"
                  "2.**Pattern identified**\n -(explain the hidden logic)\n"
                  "3.**Step by step calculation**\n -For(input values)\n   (finding the answer)"
                  "4.**Final answer**\n     -(Answer=X)\n " )


  response = client.chat.completions.create(
      model="deepseek-r1-distill-llama-70b",
      messages=[{"role":"user", "content": puzzle_prompt}],
      max_tokens=2000,
      temperature=0

  )

  if response and response.choices:
    return response.choices[0].message.content.strip()
  else:
    return "No response from the model"




In [3]:
from PIL import Image
img=Image.open("/content/iamge1.jpg")

In [4]:
print(solve_puzzle(img))

Extracted text= 2 + 7 = 27 4+4 =24 5+ 9=42 6+ 0 = ?
Refined text= 2 + 7  =  27 4+4  = 24 5+ 9 = 42 6+ 0  =  ?
<think>
Okay, so I've got this math puzzle here, and I need to figure out the pattern to solve it. Let me look at the equations provided:

2 + 7 = 27  
4 + 4 = 24  
5 + 9 = 42  
6 + 0 = ?

Hmm, at first glance, these don't seem to follow the usual addition rules because 2 + 7 is 9, not 27. So there must be some other pattern or rule that I'm missing.

Let me break it down. Maybe it's not just simple addition. Perhaps each number is being manipulated in some way before adding. Let's take the first equation: 2 + 7 = 27.

If I look at the digits separately, maybe I can find a pattern. Let's see:

- For 2 + 7: 2 and 7. Maybe 2 squared plus 7? 2² is 4, plus 7 is 11. That's not 27.
- Or maybe 2 multiplied by 7: 2*7=14. Still not 27.
- Wait, 2^3 is 8, and 7 is 7. 8 + 7 is 15. Not 27.
- Maybe 2 + 7 equals 9, and then 9 multiplied by 3 is 27. But why 3? That seems arbitrary.

Let me try

In [5]:
img3=Image.open("/content/image4.jpeg")
print(solve_puzzle(img3))


Extracted text= Simple Math Puzzle 1+4=5 2 + 5 = 12 3 + 6 = 21 8 + 11 = ?
Refined text= Simple Math Puzzle 1+4 = 5 2 + 5  =  12 3 + 6  =  21 8 + 11  =  ?
<think>
Okay, so I've got this math puzzle here, and I need to figure out the pattern to solve it. Let me look at the equations given:

1 + 4 = 5  
2 + 5 = 12  
3 + 6 = 21  
8 + 11 = ?

Hmm, at first glance, these don't seem to follow the usual addition rules. Let me check each one to see if there's a hidden pattern.

Starting with the first equation: 1 + 4 equals 5. Well, that's straightforward because 1 plus 4 is indeed 5. So, nothing unusual there.

Moving on to the second equation: 2 + 5 equals 12. Wait a minute, 2 plus 5 is 7, not 12. That's different. Maybe there's another operation involved besides addition. Let me think. Could it be multiplication? 2 times 5 is 10, which is still not 12. Hmm.

What if I try adding the numbers and then doing something else? Let's see: 2 + 5 is 7. Maybe adding the two numbers and then adding the

In [6]:
import gradio as gr

interface = gr.Interface(
    fn = solve_puzzle,
    inputs = gr.Image(type = "pil"),
    outputs = "text",
    title = "Logic Puzzle Solver",
    description = "Upload an image of a logic puzzle and get the solution."
)
interface.launch(debug="True")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9de889d7d5d2cf1d3c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9de889d7d5d2cf1d3c.gradio.live


In [7]:
with gr.Blocks(theme="soft") as interface:
    gr.Markdown(
        """
        # 🧩 Logic Puzzle Solver
        Upload an image of a logic puzzle and let AI solve it for you!
        """
    )

    with gr.Row():
        with gr.Column():
            puzzle_input = gr.Image(label="Upload Puzzle", type="pil")
            solve_button = gr.Button("Solve Puzzle")
        with gr.Column():
            result = gr.Textbox(label="Solution", placeholder="The solution will appear here.")

    solve_button.click(fn=solve_puzzle, inputs=puzzle_input, outputs=result)

interface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4e0c29dd637d985893.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Extracted text= 2 + 7 = 27 4+4 =24 5+ 9=42 6+ 0 = ?
Refined text= 2 + 7  =  27 4+4  = 24 5+ 9 = 42 6+ 0  =  ?
Extracted text= Simple Math Puzzle 1+4=5 2 + 5 = 12 3 + 6 = 21 8 + 11 = ?
Refined text= Simple Math Puzzle 1+4 = 5 2 + 5  =  12 3 + 6  =  21 8 + 11  =  ?
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4e0c29dd637d985893.gradio.live
